# Part 1의 목표
  - 경사하강법
  - 다중 선형 회귀

# 경사하강법
- gradient descent(= steepest descent)
- first-order iterative optimization algorithm for finding a local minimum of a differentiable function.
- 미분 가능한 함수의 최소값을 근사하기 위한 알고리즘의 일종
- 입력값(x)을 조금씩 변경해가며 함수값을 최소로 만드는(y가 최소가 되는) 값을 찾아가는데, 이때 x값을 증가시킬지 감소시킬지를 미분값을 통해 파악한다.
- 학습의 목적: 가능한 한 최적의 W를 찾는 것이며, 실제로는 많은 수의 가중치 및 바이어스에 대해 최적화를 해야하며 손실 함수도 복잡한 형태인 경우가 많으므로 그래프를 정확히 알 수 없음.
- 현재의 모델 변수 값(W, b)에 대한 손실 함수 값을 근거로 삼아 조금씩 손실 함수 값을 감소시키는 방향으로 학습이 진행됨.
- 문제) cost(W)를 감소시키기 위해서 어디로 가야하는지 어떻게 알 수 있나?
  - cost(W)의 미분값이 양수이면, W를 감소시킨다.
  - cost(W)의 미분값이 음수이면, W를 증가시킨다.
  - 최소값 및 극소값인 경우, 미분값이 0이 됨.
  - 경사하강법의 업데이트 수식
    - W = W - a*∂cost(W)/∂W
- 극값에 가까워질수록 점점 조금씩 움직여야 제대로 도달할 수 있음.
  - 같은 학습률을 쓰더라도 극값에 다가갈수록 미분값 자체의 절대값이 작아지므로 자연스럽게 해결됨.
- 학습률이 너무 높을 경우, 수렴하지 못하고 발산함.
- 학습률이 너무 낮을 경우, 최소값이 아닌 국소 최소값(local minimum)에 빠지게 되어 더 좋은 성능을 낼 수 있는 기회를 살리지 못함.

# 선형회귀
- 일반적인 머신러닝/딥러닝 학습 과정
  - 1. 데이터 셋 생성 혹은 로드
    - x_train = torch.FloatTensor([1,2,3]).view(-1,1)
    - y_train = torch.FloatTensor([4,8,12]).view(-1,1)

  - 2. 모델 생성 및 초기화
    - W = torch.zeros(1, requires_grad=True)
    - b = torch.zeros(1, requires_grad=True)
    - opimizer = optim.SGD([W,b], lr=0.01) # optimizer 설정
    - nb_epochs = 1000 # 원하는 만큼 경사 하강법을 반복

  - 3. 학습 루프
    - 1. 예측값 도출
      - for epoch in range(nb_epochs + 1):
          - hypo = x_train * W + b # 예측값 도출, 가설 = 예측값
    - 2. 손실 함수값 계산
          - cost = torch.mean((hypo - y_train)**2)
    - 3. 미분값 계산
          - optimizer.zero_grad() # 미분값 초기화 -> 초기화를 하지 않으면 미분값이 계속 중첩됨. 따라서 업데이트 후 필요없어진 미분값에 대해선 반드시 zero_grad()로 지워주어야 함.
          - cost.backward() # 미분값 계산
    - 4. 경사하강법 업데이트 수행
          - optimizer.step()
    - 5. 정해진 반복 횟수에 도달하거나 학습 종료 조건 만족시 break
          - if epoch % 100 == 0:
            - print('Epoch {:4d}/{} W: {:.3f}, b: {:.3f} Cost: {:.6f}'.format(epoch, nb_epochs, W.item(), b.item(), cost.item()))

- torch.optim에는 최적화를 위한 다양한 클래스가 구현되어 있으며, PyTorch에서는 이와 같이 최적화를 담당하는 객체를 optimizer라고 함.
- SGD는 Stochastic(추측 통계학) Gradient Descent의 줄임말로 가장 기본적인 optimizer이다.
  - optimizer = optim.SGD([W,b], lr = 0.01)
    - 최적화를 수행할 대상 변수들을 첫번째 매개변수로 입력하고, 학습률을 lr이라는 이름의 매개변수로 지정함.
- 이 이후, .zero_grad(), .backward(), .step() 등의 메서드들을 호출하여 최적화를 수행하게 됨.
  - zero_grad(): 미분값 초기화
  - backward(): 현재 손실 함수값을 바탕으로 미분값 계산
  - step(): 현재 미분값 및 학습률을 바탕으로 변수 업데이트 1회 수행

# 다중 선형 회귀
- 다중 선형회귀: 여러 개의 독립 변수에 대한 선형회귀
- 선형 회귀에선 하나의 독립 변수 x에만 의해서 y가 결정되었지만, 다양한 독립 변수들에 의해서 y가 결정된다.
- 독립변수들과 y의 관계를 선형으로 가정함.
- y = W1*x1 + W2*x2 + W3*x3 + b
- 다중 선형회귀에서 벡터 형태의 W 및 스칼라 형태의 b로 변수들을 생성한 후, 행렬 행태의 학습 데이터에 대해서 행렬 연산으로 간편하게 예측값을 계산해낼 수 있다.
  - b: 브로드캐스팅 기능 활용
  - 이와 같이 구현하지 않으면, W1 ~ W3 각각에 대해 변수를 생성하여야 하고 독립변수 숫자가 늘어나면 더욱 번거로워짐.

In [1]:
import torch

In [2]:
# 데이터 셋 생성 및 W, b 선언

# 1. 데이터 셋 생성
x_train = torch.FloatTensor([[73, 80,75],
                             [93, 88, 93],
                             [89, 91, 80],
                             [96, 98, 100]])
y_train = torch.FloatTensor([[152],[185],[180],[196]])

# 2. 모델 생성 및 초기화
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# 3. optimizer 설정
optimizer = torch.optim.SGD([W,b], lr=1e-5)
nb_epochs = 1000

In [3]:
# 4. 학습 루프: 행렬 연산을 이용한 예측값 계산
for epoch in range(nb_epochs + 1):
  # 가설 및 손실 함수
  hypo = x_train.matmul(W) + b # 행렬 연산 필요성: 행렬 형태의 데이터 셋, 변수 생성 및 연산을 활용하지 않으면 각 독립 변수마다 그리고 각 가중치마다 변수를 따로 생성해야 하며 가설 수식도 더 복잡해짐.
  cost = torch.mean((hypo - y_train)**2)
  # 경사하강법
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()
  # 100번마다 로그 출력
  w_list = W.view(-1).tolist()
  if epoch % 100 == 0:
    print(f'Epoch {epoch:4d} / {nb_epochs} w1: {w_list[0]:.3f}, w2: {w_list[1]:.3f}, w3: {w_list[2]:.3f}, b: {b.item():.3f} Cost: {cost.item():.6f}')

Epoch    0 / 1000 w1: 0.316, w2: 0.320, w3: 0.313, b: 0.004 Cost: 32036.250000
Epoch  100 / 1000 w1: 0.684, w2: 0.688, w3: 0.652, b: 0.008 Cost: 6.938233
Epoch  200 / 1000 w1: 0.694, w2: 0.693, w3: 0.637, b: 0.008 Cost: 6.573693
Epoch  300 / 1000 w1: 0.704, w2: 0.698, w3: 0.622, b: 0.008 Cost: 6.232265
Epoch  400 / 1000 w1: 0.714, w2: 0.702, w3: 0.608, b: 0.008 Cost: 5.912263
Epoch  500 / 1000 w1: 0.724, w2: 0.706, w3: 0.594, b: 0.008 Cost: 5.612178
Epoch  600 / 1000 w1: 0.733, w2: 0.710, w3: 0.581, b: 0.008 Cost: 5.330629
Epoch  700 / 1000 w1: 0.743, w2: 0.713, w3: 0.568, b: 0.008 Cost: 5.066350
Epoch  800 / 1000 w1: 0.752, w2: 0.717, w3: 0.555, b: 0.008 Cost: 4.818101
Epoch  900 / 1000 w1: 0.761, w2: 0.720, w3: 0.543, b: 0.008 Cost: 4.584816
Epoch 1000 / 1000 w1: 0.769, w2: 0.722, w3: 0.532, b: 0.008 Cost: 4.365375


In [4]:
# 추론
test_data = torch.FloatTensor([[73, 66, 70]])
prediction = test_data.matmul(W) + b
print(prediction)

tensor([[141.0700]], grad_fn=<AddBackward0>)


# Part 2 목표
- torch.nn.Module을 이용하여 클래스 형태로 다중 선형 회귀 구현

# torch.nn.Module
  - PyTorch에서 모든 신경망 모델의 base class가 되는 클래스
  - PyTorch에서 신경망 클래스를 구현할 때는 torch.nn.Module을 상속받아야 함.
  - Module은 또 다른 Module을 attribute로 가질 수 있음.


In [3]:
# torch.nn.Module을 이용한 다중 선형 회귀 모델
# 신경망 구성을 위한 다양한 클래스가 포함되어 있는 torch.nn은 주로 nn이라는 별칭으로 import하는 경우가 많음.

import torch
import torch.nn as nn

In [4]:
class MultipleLinearRegressionModel(nn.Module): # nn.Module을 상속받았다는 것을 알 수 있다.
  def __init__(self): # 생성자 함수
    super().__init__() # super(): base class의 메서드를 활용한다는 의미 -> 먼저 base class인 nn.Module의 생성자 함수를 호출한 다음 해당 모델에 필요한 가중치 및 편향 등을 생성해주어야 함.
    self.linear = nn.Linear(3,1) # 3개의 독립 변수로부터 1개의 값을 예측한다는 의미(간단한 신경망), y = w1*x1 + w2*x2 + w3*x3 + b
    # nn.Linear: 전통적인 fully connected neural network를 구성할 때 사용하는 클래스
    # 기본적으로 nn.Linear(입력 크기, 출력 크기) 형태로 호출

  # nn.Module에서 forward라는 메서드는 해당 모델에 입력이 들어왔을 때, 어떻게 계산해서 결과값을 만들고 반환할지를 명시하는 메서드임.
  # 오버라이딩이기 때문에 원하는대로 매개변수 개수를 설정할 수 있으나, 현재는 텐서 하나만을 x라는 이름으로 받음.
  def forward(self, x): # nn.Module 객체에서 변수명(입력값) 형태로 호출하면, forward 메서드가 호출됨.
    return self.linear(x) # self.linear(x): __init__에서 생성된 self.linear객체의 forward 메서드를 호출한다.

In [5]:
# torch.nn.functional은 인공지능 프로그래밍에 쓰이는 다양한 함수들을 포함하고 있으며 대표적인 손실 함수들이 있음.
import torch.nn.functional as F
# MSE의 경우에는 mse_loss라는 이름의 함수로 존재하며, 사용할 때는 mse_loss(예측값, 레이블) 형태로 호출하면 됨.
# cost = F.mse_loss(prediction, y_train): 평균 제곱 오차(MSE)

In [6]:
# nn.Module에서 parameters() 메서드를 호출하면, 내부의 모든 가중치 및 편향을 모아서 반환함.
# W, b를 직접 선언할 때와는 달리 nn.Module을 이용한 클래스 형태로 모델을 구현하게 되면, optimizer 선언 시 parameters() 메서드를 활용해서 최적화를 진행할 변수들을 입력함.
# PyTorch에서는 가중치 및 편향을 초기화하는 default 방식이 따로 존재하기 때문에 초기화된 상태로 출력해도 zero tensor가 아닌 것을 확인할 수 있음.
model = MultipleLinearRegressionModel()
print(list(model.parameters())) # W 3개와 b 1개 반환

optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

[Parameter containing:
tensor([[-0.2276, -0.1599,  0.2143]], requires_grad=True), Parameter containing:
tensor([0.3560], requires_grad=True)]


In [7]:
# optimizer in Pytorch: 모델의 loss 값을 줄이는 방향으로 parameters를 조절함.
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [8]:
# 모델 코드
class MultipleLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3,1)
  def forward(self, x):
    return self.linear(x)

In [9]:
# 학습 코드
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 80],
                             [96, 98, 100]])

y_train = torch.FloatTensor([[152], [185], [180], [196]])

In [11]:
model = MultipleLinearRegressionModel()
print(list(model.parameters()))

[Parameter containing:
tensor([[ 0.1146, -0.2252, -0.2295]], requires_grad=True), Parameter containing:
tensor([-0.4147], requires_grad=True)]


In [14]:
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)
nb_epochs = 1000

In [20]:
for epoch in range(nb_epochs + 1):
  prediction = model(x_train)
  cost = F.mse_loss(prediction, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print(f'Epoch: {epoch} / {nb_epochs} Cost: {cost.item():.6f}')

Epoch: 0 / 1000 Cost: 83.152328
Epoch: 100 / 1000 Cost: 4.003900
Epoch: 200 / 1000 Cost: 3.755467
Epoch: 300 / 1000 Cost: 3.524437
Epoch: 400 / 1000 Cost: 3.309574
Epoch: 500 / 1000 Cost: 3.109640
Epoch: 600 / 1000 Cost: 2.923544
Epoch: 700 / 1000 Cost: 2.750206
Epoch: 800 / 1000 Cost: 2.588792
Epoch: 900 / 1000 Cost: 2.438357
Epoch: 1000 / 1000 Cost: 2.298064


In [21]:
# 추론
test_data = torch.FloatTensor([[73, 66, 70]])
prediction = model(test_data)
print(prediction)

tensor([[142.0052]], grad_fn=<AddmmBackward0>)
